# BEHAVIOR Prep 1 — Frame Debugging & Transform Order (Applied)
**Aim:** Instantly diagnose wrong-order / wrong-inversion bugs in 3D robotics pipelines.

Designed like a real ticket: *an object looks offset/mirrored/rotated — find the transform bug*.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
np.set_printoptions(precision=4, suppress=True)


In [ ]:
def skew(w):
    wx, wy, wz = w
    return np.array([[0, -wz, wy],
                     [wz, 0, -wx],
                     [-wy, wx, 0]], float)

def make_T(R, p):
    T = np.eye(4)
    T[:3,:3] = R
    T[:3,3] = p
    return T

def inv_T(T):
    R = T[:3,:3]
    p = T[:3,3]
    Rt = R.T
    return make_T(Rt, -Rt @ p)

def rotz(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]], float)

def roty(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,0,s],[0,1,0],[-s,0,c]], float)

def plot_frame(ax, T, name="", L=0.12):
    o = T[:3,3]
    R = T[:3,:3]
    cols = ["r","g","b"]
    for i in range(3):
        ax.plot([o[0], o[0]+R[0,i]*L],
                [o[1], o[1]+R[1,i]*L],
                [o[2], o[2]+R[2,i]*L], c=cols[i], linewidth=2)
    if name:
        ax.text(o[0], o[1], o[2], " "+name)

def set_axes_equal(ax, lim=1.0):
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(-lim, lim)
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")


## 1) Transform meaning
Convention used here:

- **`T_AB` maps coordinates in frame B → into frame A**

So a point `p_B` becomes `p_A = T_AB @ p_B` (homogeneous).


In [ ]:
import numpy as np

p_B = np.array([0.2, 0.0, 0.0, 1.0])
theta = np.deg2rad(45)
T_AB = make_T(rotz(theta), np.array([0.1, 0.05, 0.0]))
p_A = T_AB @ p_B

print("T_AB=\n", T_AB)
print("p_A =", p_A)


## 2) Ticket: chain camera→object→world
**Correct:** `T_wo = T_wc @ T_co`

We also create a wrong-order chain to show the bug.


In [ ]:
T_wc = make_T(rotz(np.deg2rad(-30)), np.array([0.8, -0.2, 0.6]))   # c -> w
T_co = make_T(roty(np.deg2rad(20)),  np.array([0.25, 0.05, 0.30])) # o -> c

T_wo_correct = T_wc @ T_co
T_wo_bug = T_co @ T_wc  # WRONG order

print("Correct object position in world:", T_wo_correct[:3,3])
print("Buggy  object position in world:", T_wo_bug[:3,3])


## 3) Visualize frames


In [ ]:
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')

plot_frame(ax, np.eye(4), "w")
plot_frame(ax, T_wc, "c")
plot_frame(ax, T_wo_correct, "o(correct)")
plot_frame(ax, T_wo_bug, "o(bug)")

set_axes_equal(ax, lim=1.2)
plt.title("World w, Camera c, Object o (correct vs buggy chain)")
plt.show()


## 4) Inversion bug demo


In [ ]:
T_oc = inv_T(T_co)                 # c -> o
T_wo_from_oc = T_wc @ inv_T(T_oc)  # should match T_wo_correct

print("||difference|| =", np.linalg.norm(T_wo_from_oc - T_wo_correct))
